In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.matrix as smatrix
%matplotlib inline

The Health Department has developed an inspection report and scoring system. 
After conducting an inspection of the facility, the Health Inspector calculates a score based on the violations observed. 
Violations can fall into:
- **high risk category**: records specific violations that directly relate to the transmission of food borne illnesses,the adulteration of food products and the contamination of food-contact surfaces.
- **moderate risk category**: records specific violations that are of a moderate risk to the public health and safety.
- **low risk category**: records violations that are low risk or have no immediate risk to the public health and safety.

In [ ]:
businesses = pd.read_csv('./data/businesses_plus.csv', parse_dates=True, dtype={'phone_number': str})
businesses.head()
# dtype casts the column as a specific data type

In [ ]:
inspections = pd.read_csv('./data/inspections_plus.csv', parse_dates=True)
inspections['inspections_id']=inspections.index
inspections.head()


In [ ]:
violations = pd.read_csv('./data/violations_plus.csv', parse_dates=True)
violations['violations_id']=violations.index

violations.head()

In [ ]:
# 1 Combine the three dataframes into one data frame called restaurant_scores
# Hint: http://pandas.pydata.org/pandas-docs/stable/merging.html

inspvio = pd.merge(inspections, violations, 
                   on=['business_id', 'date'])

inspvio.head()

In [ ]:
df = pd.merge(businesses, inspvio, on=['business_id'])

df.head()

In [ ]:
print businesses.shape
print inspections.shape
print violations.shape
print inspvio.shape
print df.shape

In [ ]:
# 2 Which ten business have had the most inspections?
#inspections.business_id.value_counts().head(10)


#movie_ratings.groupby('title').rating.mean().order(ascending=False)
#drinks[['country', 'beer_servings']].head()

df.groupby('name').inspections_id.nunique().nlargest(10)


In [ ]:
# 3 Group and count the inspections by type


inspections.type.value_counts().sort_values(ascending=False)


In [ ]:
# 4 Create a plot that shows number of inspections per month
# criar coluna que trunca data em mes. 
# plotar por essa coluna
inspections['monthinsp']=inspections['date'].astype(str).apply(lambda x: x[:6]).astype(int)
df['monthdf']=df['date'].astype(str).apply(lambda x: x[:6]).astype(int)
inspections.monthinsp.head()

# Bonus for creating a heatmap
# http://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.heatmap.html?highlight=heatmap

In [ ]:
#wrong, please ignore
#inspections.groupby('monthinsp').count().plot(kind='bar', title='Inspections per Month')
#plt.xlabel('Month')
#plt.ylabel('Count')


In [ ]:
df.groupby('monthdf').inspections_id.nunique().plot(kind='bar', title='Inspections per Month')
plt.xlabel('Month')
plt.ylabel('Count')

In [ ]:
data = pd.DataFrame(df.groupby('monthdf').inspections_id.nunique())

sns.heatmap(data)

In [ ]:
# 5 Which zip code contains the most high risk violations?

df.groupby('postal_code').violations_id.nunique().nlargest(10)

In [ ]:
# 6 If inspection is prompted by a change in restaurant ownership, 
# is the inspection more likely to be categorized as higher or lower risk?
df[df.type=='New Ownership']['risk_category'].value_counts(normalize=True)

In [ ]:
# 7 Examining the descriptions, what is the most common violation?

violations.description.value_counts().head(30)


In [ ]:
# 8 Create a hist of the scores with 10 bins

df.Score.hist(bins=10)

In [ ]:
# 9 Can you predict risk category based on the other features in this dataset?
df['risk_category_number'] = df.risk_category.map({'Low Risk': 1, 'Moderate Risk': 2, 'High Risk': 3})
df.head()

In [ ]:
df.corr().risk_category_number

In [ ]:
#Score and ViolationTypeID seem to be the best ones to predict risk_category

In [ ]:
df['dirty_calculated_risk'] = 100000 + df['Score']*-0.244403 + df['ViolationTypeID']*-0.922796
df.head()

In [ ]:
df.groupby('name').dirty_calculated_risk.mean().head()

In [ ]:
topname = df.groupby('name').dirty_calculated_risk.mean().index.tolist()
topname[0]

In [ ]:
df[['latitude','longitude']][df.name==topname[0]].head(1)

In [ ]:
# 10 Extra Credit:
# Use Instagram location API to find pictures taken at the lat, long of the most High Risk restaurant
# https://www.instagram.com/developer/endpoints/locations/

# Need Sinan's help with the access_token for IG.
# https://api.instagram.com/v1/locations/search?lat=37.784317&lng=-122.407563&access_token=
#    fb2e77d.47a0479900504cb3ab4a1f626d174d2d


In [ ]:
############################
### A Little More Morbid ###
############################

In [ ]:
killings = pd.read_csv('./data/police-killings.csv')
killings.head()

In [ ]:
# 1. Make the following changed to column names:
# lawenforcementagency -> agency
# raceethnicity        -> race

killings = killings.rename(columns={'lawenforcementagency': 'agency', 'raceethnicity': 'race'})
killings.head()

In [ ]:
# 2. Show the count of missing values in each column

killings.isnull().sum()  

In [ ]:
# 3. replace each null value in the dataframe with the string "Unknown"

killings.fillna(value='Unknown', inplace=True) 

In [ ]:
killings.isnull().sum()  

In [ ]:
# 4. How many killings were there so far in 2015?

killings[killings.year == 2015].year.value_counts()

In [ ]:
# 5. Of all killings, how many were male and how many female?


killings.gender.value_counts()

In [ ]:
# 6. How many killings were of unarmed people?
killings[killings.armed == 'No'].armed.value_counts()

In [ ]:
# 7. What percentage of all killings were unarmed?
killings.armed.value_counts()

In [ ]:
# 8. What are the 5 states with the most killings?
killings.state.value_counts().sort_values(ascending=False).head()

In [ ]:
# 9. Show a value counts of deaths for each race
killings.race.value_counts()

In [ ]:
# 10. Display a histogram of ages of all killings
killings.age.hist(bins=20)

In [ ]:
# 11. Show 6 histograms of ages by race
killings.age.hist(by=killings.race)


In [ ]:
# 12. What is the average age of death by race?

killings.groupby('race').age.mean()

In [ ]:
# 13. Show a bar chart with counts of deaths every month
killings.month.value_counts().plot(kind='bar', title='Deaths per Month in 2015')


In [ ]:
###################
### Less Morbid ###
###################

In [ ]:
majors = pd.read_csv('./data/college-majors.csv')
majors.head()

In [ ]:
# 1. Delete the columns (employed_full_time_year_round, major_code)

del majors['Employed_full_time_year_round']
del majors['Major_code']    

In [ ]:
majors.head()

In [ ]:
# 2. Show the cout of missing values in each column

majors.isnull().sum()

In [ ]:
# 3. What are the top 10 highest paying majors?
majors[['Major','P75th']].sort_values(by='P75th', ascending=False).head(10)

In [ ]:
# 4. Plot the data from the last question in a bar chart, include proper title, and labels!

#drinks.groupby('continent').beer_servings.mean().plot(kind='bar')


majors.groupby('Major').P75th.mean().sort_values(ascending=False).head(10).plot(kind='bar', title='P75th by major')
plt.xlabel('Major')
plt.ylabel('Salary')

In [ ]:
# 5. What is the average median salary for each major category?
majors.groupby('Major_category').Median.mean()

In [ ]:
# 6. Show only the top 5 paying major categories
majors.groupby('Major_category').Median.mean().head()

In [ ]:
# 7. Plot a histogram of the distribution of median salaries
majors.Median.hist(bins=20)

In [ ]:
# 8. Plot a histogram of the distribution of median salaries by major category
majors.groupby('Major_category').Median.hist(bins=20)

In [ ]:
# 9. What are the top 10 most UNemployed majors?

majors[['Major','Unemployment_rate']].sort_values(by='Unemployment_rate', ascending=False).head(10)

In [ ]:
# What are the unemployment rates?
# Answered above

In [ ]:
# 10. What are the top 10 most UNemployed majors CATEGORIES? Use the mean for each category
# What are the unemployment rates?
majors.groupby('Major_category')['Unemployment_rate'].mean().sort_values(ascending=False).head(10)

In [ ]:
# 11. the total and employed column refer to the people that were surveyed.
# Create a new column showing the emlpoyment rate of the people surveyed for each major
# call it "sample_employment_rate"
# Example the first row has total: 128148 and employed: 90245. it's 
# sample_employment_rate should be 90245.0 / 128148.0 = .7042

majors['sample_employment_rate'] = majors.Employed / majors.Total


In [ ]:
# 12. Create a "sample_unemployment_rate" column
# this column should be 1 - "sample_employment_rate"

majors['sample_unemployment_rate'] = 1 - majors.sample_employment_rate


In [ ]:
majors.head()